<a href="https://colab.research.google.com/github/dlwltn0430/autonomous-driving-samples/blob/main/car_speed_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras opencv-python-headless

In [ ]:
import cv2
import numpy as np
import time
from google.colab.patches import cv2_imshow

# 차량 이동 거리 계산을 위한 함수
def calculate_distance(p1, p2):
    return np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

# 차량 속도 계산을 위한 함수
def calculate_speed(distance, time_elapsed):
    return (distance / time_elapsed) * 3.6  # m/s를 km/h로 변환

# 비디오 캡처 시작
cap = cv2.VideoCapture('driving_video.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

# 첫 번째 프레임 읽기
ret, frame1 = cap.read()
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
prev_points = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

speeds = []
prev_time = time.time()

while cap.isOpened():
    ret, frame2 = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None)

    # 유효한 포인트만 선택
    good_new = next_points[status == 1]
    good_old = prev_points[status == 1]

    # 각 프레임 간의 속도 계산
    distances = [calculate_distance(p1, p2) for p1, p2 in zip(good_old, good_new)]
    avg_distance = np.mean(distances)

    current_time = time.time()
    time_elapsed = current_time - prev_time
    speed = calculate_speed(avg_distance, time_elapsed)
    speeds.append(speed)

    # 프레임 업데이트
    prev_gray = gray.copy()
    prev_points = good_new.reshape(-1, 1, 2)
    prev_time = current_time

    # 속도 표시
    cv2.putText(frame2, f'Speed: {speed:.2f} km/h', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2_imshow(frame2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 평균 속도 계산 및 출력
average_speed = np.mean(speeds)
print(f'Average Speed: {average_speed:.2f} km/h')